# It's a Machine and Natural Language Tagger

In [1]:
from src.IaMaN.base import LM
from src.utils.data import load_ud
from src.utils.munge import stick_spaces
from pprint import pprint as pprint
from collections import defaultdict
from collections import Counter
from tqdm import tqdm
import numpy as np
import os, re

seed = 691; max_char = 200_000_000
m = 10; space = True; fine_tune = False; num_posttrain = 1000; noise = 0.001
positional = 'dependent'; positionally_encode = True; bits = 50; btype = 'all'
ms_init = 'waiting_time'; update = {'representation', 'contexts', 'tokenizer'}
runners = 10; gpu = False; tokenizer = 'hr-bpe'; decode_method = 'argmax'

print("Loading post-training data...")
posttrain_path = '/data/newstweet/week_2019-40_article_texts/'
total_posttrain = len([posttrain_file for posttrain_file in os.listdir(posttrain_path) if re.search("^\d+.txt$", posttrain_file)])
all_posttrain_files = [posttrain_file for posttrain_file in os.listdir(posttrain_path) if re.search("^\d+.txt$", posttrain_file)]
if num_posttrain:
    np.random.seed(seed)
    posttrain_files = np.random.choice(all_posttrain_files, size=num_posttrain, replace=False)
else:
    posttrain_files = np.array([])
ptdocs = [[open(posttrain_path+posttrain_file).read()] for posttrain_file in tqdm(posttrain_files)]
print("Loading gold-tagged UDs data...")
load_set = "GUM"
all_docs = load_ud("English", num_articles = 0, seed = seed, load_set = load_set, rebuild = True, space = space)
test_docs = [doc for doc in all_docs if 'test' in doc['id'] and len(doc['text']) <= max_char]# [:2]
train_docs = [doc for doc in all_docs if 'test' not in doc['id'] and len(doc['text']) <= max_char]# [:4]
nsamp = len(test_docs)
print('Avail. post-train, total post-train, Avail. gold, total gold-train, total test-gold: ', 
      total_posttrain, len(ptdocs), len(all_docs), len(train_docs), len(test_docs))

Loading post-training data...


100%|██████████| 1000/1000 [00:02<00:00, 473.82it/s]


Loading gold-tagged UDs data...
Avail. post-train, total post-train, Avail. gold, total gold-train, total test-gold:  14198 1000 150 132 18


In [2]:
docs = [["".join([row[1] for row in s]) for s in d['conllu']] for d in train_docs]
tdocs = [["".join([row[1] for row in s]) for s in d['conllu']] for d in test_docs]
covering = [[[row[1] for row in s] for s in d['conllu']] for d in train_docs]
tcovering = [[[row[1] for row in s] for s in d['conllu']] for d in test_docs]
if not space:
    for d_i, d in enumerate(covering):
        for s_i, s in enumerate(d):
            covering[d_i][s_i] = stick_spaces(s)
    for d_i, d in enumerate(tcovering):
        for s_i, s in enumerate(d):
            tcovering[d_i][s_i] = stick_spaces(s)
covering_vocab = set([t for d in covering for s in d for t in s])

# 'lem': [[row[2] for row in s] for s in d['conllu']], # note: for speed, remove lemma layer
train_layers = {d_i: {'sty': [[d['s_type'][s_i] for row in s] for s_i, s in enumerate(d['conllu'])], 
                      'pos': [[row[3] for row in s] for s in d['conllu']], 
                      'sup': [[(str(int(row[6]) - int(row[0])) if int(row[6]) else row[6]) for row in s] for s in d['conllu']], 
                      'dep': [[row[7] for row in s] for s in d['conllu']]}
                for d_i, d in enumerate(train_docs)}
# 'lem': [[row[2] for row in s] for s in d['conllu']], # note: for speed, remove lemma layer
test_layers = {d_i: {'sty': [[d['s_type'][s_i] for row in s] for s_i, s in enumerate(d['conllu'])], 
                     'pos': [[row[3] for row in s] for s in d['conllu']], 
                     'sup': [[(str(int(row[6]) - int(row[0])) if int(row[6]) else row[6]) for row in s] for s in d['conllu']], 
                     'dep': [[row[7] for row in s] for s in d['conllu']]}
               for d_i, d in enumerate(test_docs)}

model = LM(m = m, tokenizer = tokenizer, noise = noise, seed = seed, space = space, positional = positional,
           positionally_encode = positionally_encode, runners = runners, gpu = gpu, bits = bits, 
           btype = btype, ms_init = ms_init)
docs_name = f'{load_set}-{len(train_docs)}'
ptdocs_name = f'newstweet-{num_posttrain}'
data_streams = model.fit(docs, docs_name, covering = covering, all_layers = train_layers, fine_tune = fine_tune)

Training tokenizer


Fitting:  20%|██        | 20/100 [00:43<02:53,  2.17s/it]


Built a vocabulary of 15327 types
Pre-processing data...


Counting tag-tag transition frequencies: 100%|██████████| 132/132 [00:00<00:00, 224.73it/s]


Counting documents and aggregating counts...


Building dense decoders: 100%|██████████| 23/23 [00:00<00:00, 32.24it/s]
Building transition matrices for tag-sequence decoding:   0%|          | 0/7 [00:00<?, ?it/s]/code/IaMaN/src/IaMaN/base.py:673: RuntimeWarning: invalid value encountered in true_divide
  self._trXs[ltype] = (lambda M: M/M.sum(axis = 1)[:,None])(self._trXs[ltype])
Building transition matrices for tag-sequence decoding: 100%|██████████| 7/7 [00:00<00:00, 3097.39it/s]

Model params, types, encoding size, contexts, vec dim, max sent, and % capacity used: 886810 10568 50 1071 353 258 9.679


In [3]:
model._ms

{'form': 8,
 'bits': 7,
 'sty': 2,
 'pos': 6,
 'sup': 7,
 'dep': 7,
 'nov': 2,
 'iat': 2,
 'bot': 2,
 'eot': 2,
 'eos': 2,
 'eod': 2}

In [4]:
model.interpret([[' Gone again!']], seed = seed)
for doc in model._documents:
    print('opening next doc:')
    for s in doc._sentences:
        print(f'opening next sent: {s._sty}')
        for t in s._tokens:
            print(f'opening next token: {t._form}, {t._lem}, {t._sep}, {t._pos}, {t._sup}, {t._dep}')
            print(["\""+w._form+"\"" for w in t._whatevers])

Pre-processing data...


Interpreting documents: 100%|██████████| 1/1 [00:00<00:00, 155.33it/s]


opening next doc:
opening next sent: decl
opening next token:  , None, False, SPACE, 1, space
['" "']
opening next token: Gone, None, False, X, 0, root
['"Gon"', '"e"']
opening next token:  , None, False, SPACE, -1, dep
['" "']
opening next token: again, None, False, PUNCT, -1, punct
['"again"']
opening next token: !, None, True, PUNCT, -1, punct
['"!"']


In [5]:
d_i = 3; s_max = 2
interpret_docs = list([docs[d_i][:s_max]])
print(interpret_docs)
model.interpret(interpret_docs, eval_layers = {0: {tag: train_layers[d_i][tag][:s_max] for tag in train_layers[d_i]}},
                eval_covering = [covering[d_i][:s_max]], seed = seed)

for doc in model._documents:
    print('opening next doc:')
    for s in doc._sentences:
        print(f'opening next sent: {s._sty}')
        for t in s._tokens:
            print(f'opening next token: {t._form}, {t._lem}, {t._sep}, {t._pos}, {t._sup}, {t._dep}')

[[' Emperor Norton ', ' Joshua Abraham Norton (c. 1818 – January 8, 1880), known as Emperor Norton, was a citizen of San Francisco, California, who in 1859 proclaimed himself "Norton I, Emperor of the United States". ']]
Pre-processing data...


Interpreting documents: 100%|██████████| 1/1 [00:00<00:00,  1.34it/s]

Document 0's Sentence segmentation performance:  {('R', 1.0), ('P', 1.0), ('F', 1.0)}
Document 0's STY accuracy:  0.5
Document 0's Token segmentation performance without space:  {('F', 0.9782608695652174), ('P', 0.9782608695652174), ('R', 0.9782608695652174)}
Document 0's POS accuracy with/out space: 0.7530864197530864 0.5555555555555556
Document 0's SUP:DEP accuracy with/out space:  0.4691358024691358 0.1111111111111111

Overall sentence segmentation performance:  {('R', 1.0), ('P', 1.0), ('F', 1.0)}
Overall STY accuracy:  0.5
Overall Token segmentation performance without space:  {('F', 0.9782608695652174), ('P', 0.9782608695652174), ('R', 0.9782608695652174)}
Overall POS accuracy with/out space: 0.7530864197530864 0.5555555555555556
Overall SUP:DEP accuracy with/out space:  0.4691358024691358 0.1111111111111111
opening next doc:
opening next sent: decl
opening next token:  , None, False, SPACE, 1, space
opening next token: Emperor, None, False, PROPN, 2, vocative
opening next token:

In [6]:
d_i = 0; s_max = 1
interpret_docs = list([tdocs[d_i][:s_max]])
print(interpret_docs)
model.interpret(interpret_docs, eval_layers = {0: {tag: test_layers[d_i][tag][:s_max] for tag in test_layers[d_i]}},
                eval_covering = [tcovering[d_i][:s_max]], seed = seed)

for doc in model._documents:
    print('opening next doc:')
    for s in doc._sentences:
        print(f'opening next sent: {s._sty}')
        for t in s._tokens:
            print(f'opening next token: {t._form}, {t._lem}, {t._sep}, {t._pos}, {t._sup}, {t._dep}')

[[' The prevalence of discrimination across racial groups in contemporary America: ']]
Pre-processing data...


Interpreting documents: 100%|██████████| 1/1 [00:00<00:00, 15.07it/s]

Document 0's Sentence segmentation performance:  {('R', 1.0), ('P', 1.0), ('F', 1.0)}
Document 0's STY accuracy:  0.0
Document 0's Token segmentation performance without space:  {('R', 1.0), ('P', 1.0), ('F', 1.0)}
Document 0's POS accuracy with/out space: 0.6818181818181818 0.36363636363636365
Document 0's SUP:DEP accuracy with/out space:  0.5454545454545454 0.2727272727272727

Overall sentence segmentation performance:  {('R', 1.0), ('P', 1.0), ('F', 1.0)}
Overall STY accuracy:  0.0
Overall Token segmentation performance without space:  {('R', 1.0), ('P', 1.0), ('F', 1.0)}
Overall POS accuracy with/out space: 0.6818181818181818 0.36363636363636365
Overall SUP:DEP accuracy with/out space:  0.5454545454545454 0.2727272727272727
opening next doc:
opening next sent: decl
opening next token:  , None, False, SPACE, 1, space
opening next token: The, None, False, PROPN, 2, vocative
opening next token:  , None, False, SPACE, 1, space
opening next token: prevalence, None, False, PROPN, 0, root

In [7]:
d_i = 1; s_max = 2
interpret_docs = list([tdocs[d_i][:s_max]])
print(interpret_docs)
model.interpret(interpret_docs, eval_layers = {0: {tag: test_layers[d_i][tag][:s_max] for tag in test_layers[d_i]}},
                eval_covering = [tcovering[d_i][:s_max]], seed = seed)
for doc in model._documents:
    print('opening next doc:')
    for s in doc._sentences:
        print(f'opening next sent: {s._sty}')
        for t in s._tokens:
            print(f'opening next token: {t._form}, {t._lem}, {t._sep}, {t._pos}, {t._sup}, {t._dep}')

[[' 2. GUJJOLAAY EEGIMAA, ITS SPEAKERS AND THEIR NEIGHBOURS ', ' This section briefly presents the Gújjolaay Eegimaa (Eegimaa for short; Ethnologue code: ISO 639-3: bqj), its speakers and its varieties. ']]
Pre-processing data...


Interpreting documents: 100%|██████████| 1/1 [00:00<00:00,  1.31it/s]

Document 0's Sentence segmentation performance:  {('R', 1.0), ('P', 1.0), ('F', 1.0)}
Document 0's STY accuracy:  0.5
Document 0's Token segmentation performance without space:  {('F', 0.6015037593984962), ('P', 0.5128205128205128), ('R', 0.7272727272727273)}
Document 0's POS accuracy with/out space: 0.6515151515151515 0.3611111111111111
Document 0's SUP:DEP accuracy with/out space:  0.3787878787878788 0.05555555555555555

Overall sentence segmentation performance:  {('R', 1.0), ('P', 1.0), ('F', 1.0)}
Overall STY accuracy:  0.5
Overall Token segmentation performance without space:  {('F', 0.6015037593984962), ('P', 0.5128205128205128), ('R', 0.7272727272727273)}
Overall POS accuracy with/out space: 0.6515151515151515 0.3611111111111111
Overall SUP:DEP accuracy with/out space:  0.3787878787878788 0.05555555555555555
opening next doc:
opening next sent: decl
opening next token:  , None, False, SPACE, 1, space
opening next token: 2, None, False, PROPN, 5, vocative
opening next token: ., 

In [8]:
d_i = 1; s_max = 2
interpret_docs = list([tdocs[d_i][:s_max]])
print(interpret_docs)
model.interpret(interpret_docs, eval_layers = {0: {tag: test_layers[d_i][tag][:s_max] for tag in test_layers[d_i]}},
                covering = [tcovering[d_i][:s_max]], seed = seed)
for doc in model._documents:
    print('opening next doc:')
    for s in doc._sentences:
        print(f'opening next sent: {s._sty}')
        for t in s._tokens:
            print(f'opening next token: {t._form}, {t._lem}, {t._sep}, {t._pos}, {t._sup}, {t._dep}')

[[' 2. GUJJOLAAY EEGIMAA, ITS SPEAKERS AND THEIR NEIGHBOURS ', ' This section briefly presents the Gújjolaay Eegimaa (Eegimaa for short; Ethnologue code: ISO 639-3: bqj), its speakers and its varieties. ']]
Pre-processing data...


Interpreting documents: 100%|██████████| 1/1 [00:00<00:00,  2.79it/s]

Document 0's STY accuracy:  0.5
Document 0's POS accuracy with/out space: 0.7121212121212122 0.4722222222222222
Document 0's SUP:DEP accuracy with/out space:  0.3484848484848485 0.0

Overall STY accuracy:  0.5
Overall POS accuracy with/out space: 0.7121212121212122 0.4722222222222222
Overall SUP:DEP accuracy with/out space:  0.3484848484848485 0.0
opening next doc:
opening next sent: decl
opening next token:  , None, False, SPACE, 1, space
opening next token: 2., None, False, PROPN, 5, vocative
opening next token:  , None, False, SPACE, 1, space
opening next token: GUJJOLAAY, None, False, PROPN, 0, root
opening next token:  , None, False, SPACE, -1, space
opening next token: EEGIMAA, None, False, PROPN, -2, punct
opening next token: ,, None, False, PUNCT, -1, punct
opening next token:  , None, False, SPACE, 1, space
opening next token: ITS, None, False, PUNCT, -3, punct
opening next token:  , None, False, SPACE, -1, space
opening next token: SPEAKERS, None, False, PROPN, -1, punct
open

In [9]:
model.interpret(tdocs, eval_layers = test_layers, eval_covering = tcovering, seed = seed, verbose_result = False, 
                decode_method = decode_method)

Pre-processing data...


Interpreting documents:   6%|▌         | 1/18 [00:15<04:15, 15.01s/it]

Document 0's Sentence segmentation performance:  {('R', 1.0), ('P', 1.0), ('F', 1.0)}
Document 0's STY accuracy:  0.6851851851851852
Document 0's Token segmentation performance without space:  {('F', 0.9181619256017505), ('P', 0.9161572052401746), ('R', 0.9201754385964912)}
Document 0's POS accuracy with/out space: 0.7676056338028169 0.560418648905804
Document 0's SUP:DEP accuracy with/out space:  0.556841046277666 0.1912464319695528



Interpreting documents:  11%|█         | 2/18 [00:35<04:52, 18.28s/it]

Document 1's Sentence segmentation performance:  {('R', 1.0), ('P', 1.0), ('F', 1.0)}
Document 1's STY accuracy:  0.8611111111111112
Document 1's Token segmentation performance without space:  {('F', 0.769850866586054), ('P', 0.7095096582466568), ('R', 0.8414096916299559)}
Document 1's POS accuracy with/out space: 0.7576112412177985 0.5394883203559511
Document 1's SUP:DEP accuracy with/out space:  0.5327868852459017 0.14015572858731926



...

Document 15's Sentence segmentation performance:  {('R', 1.0), ('P', 1.0), ('F', 1.0)}
Document 15's STY accuracy:  0.8421052631578947
Document 15's Token segmentation performance without space:  {('F', 0.8513157894736841), ('P', 0.8252551020408163), ('R', 0.8790760869565217)}
Document 15's POS accuracy with/out space: 0.7866449511400652 0.5808
Document 15's SUP:DEP accuracy with/out space:  0.5814332247557004 0.1936



Interpreting documents:  94%|█████████▍| 17/18 [03:43<00:11, 11.49s/it]

Document 16's Sentence segmentation performance:  {('R', 1.0), ('P', 1.0), ('F', 1.0)}
Document 16's STY accuracy:  0.32558139534883723
Document 16's Token segmentation performance without space:  {('F', 0.9072164948453609), ('P', 0.9004092769440655), ('R', 0.9141274238227147)}
Document 16's POS accuracy with/out space: 0.7850911974623315 0.5826893353941267
Document 16's SUP:DEP accuracy with/out space:  0.5860428231562252 0.21792890262751158



Interpreting documents: 100%|██████████| 18/18 [03:53<00:00, 12.95s/it]

Document 17's Sentence segmentation performance:  {('R', 1.0), ('P', 1.0), ('F', 1.0)}
Document 17's STY accuracy:  0.4153846153846154
Document 17's Token segmentation performance without space:  {('F', 0.908675799086758), ('P', 0.898014440433213), ('R', 0.9195933456561922)}
Document 17's POS accuracy with/out space: 0.8134981968057702 0.6365461847389559
Document 17's SUP:DEP accuracy with/out space:  0.5734157650695518 0.19477911646586346

Overall sentence segmentation performance:  {('R', 1.0), ('P', 1.0), ('F', 1.0)}
Overall STY accuracy:  0.6588366890380313
Overall Token segmentation performance without space:  {('F', 0.8694372340710215), ('P', 0.850879949717159), ('R', 0.8888220167423537)}
Overall POS accuracy with/out space: 0.7733836137433121 0.5667242229896399
Overall SUP:DEP accuracy with/out space:  0.558241474892026 0.17939072520966945


In [10]:
model.interpret(tdocs, eval_layers = test_layers, covering = tcovering, seed = seed, verbose_result = False, 
                decode_method = decode_method)

Pre-processing data...


Interpreting documents:   6%|▌         | 1/18 [00:13<03:43, 13.16s/it]

Document 0's STY accuracy:  0.6851851851851852
Document 0's POS accuracy with/out space: 0.7917505030181087 0.6060894386298763
Document 0's SUP:DEP accuracy with/out space:  0.5653923541247485 0.20456707897240722



Interpreting documents:  11%|█         | 2/18 [00:25<03:26, 12.88s/it]

Document 1's STY accuracy:  0.8611111111111112
Document 1's POS accuracy with/out space: 0.7933255269320844 0.60734149054505
Document 1's SUP:DEP accuracy with/out space:  0.5374707259953162 0.14015572858731926



...

Document 15's STY accuracy:  0.8421052631578947
Document 15's POS accuracy with/out space: 0.8281758957654723 0.6624
Document 15's SUP:DEP accuracy with/out space:  0.5806188925081434 0.2



Interpreting documents:  94%|█████████▍| 17/18 [03:03<00:09,  9.74s/it]

Document 16's STY accuracy:  0.32558139534883723
Document 16's POS accuracy with/out space: 0.8049167327517843 0.6213292117465224
Document 16's SUP:DEP accuracy with/out space:  0.5828707375099128 0.23183925811437403



Interpreting documents: 100%|██████████| 18/18 [03:11<00:00, 10.66s/it]

Document 17's STY accuracy:  0.4153846153846154
Document 17's POS accuracy with/out space: 0.8444100978876867 0.6967871485943775
Document 17's SUP:DEP accuracy with/out space:  0.5821741370427614 0.20983935742971888

Overall STY accuracy:  0.6588366890380313
Overall POS accuracy with/out space: 0.8001676013665957 0.6179699062654169
Overall SUP:DEP accuracy with/out space:  0.5643009089151034 0.1910458806117415


In [11]:
_ = model.fit(ptdocs, ptdocs_name, update = update, fine_tune = fine_tune, bits = bits)
if fine_tune and ptdocs:
    model.fine_tune(docs, covering = covering, all_layers = train_layers, streams = data_streams)

Training tokenizer


Fitting:  14%|█▍        | 14/100 [02:35<15:54, 11.10s/it]


Built a vocabulary of 41538 types
Pre-processing data...


Counting tag-tag transition frequencies: 100%|██████████| 1132/1132 [00:01<00:00, 676.40it/s]


Counting documents and aggregating counts...


Building dense decoders: 100%|██████████| 23/23 [00:00<00:00, 34.82it/s]
Building transition matrices for tag-sequence decoding: 100%|██████████| 7/7 [00:00<00:00, 3192.01it/s]

Model params, types, encoding size, contexts, vec dim, max sent, and % capacity used: 2100689 10300 50 1071 353 18310 21.048


In [12]:
if ptdocs:
    model.interpret(tdocs, eval_layers = test_layers, eval_covering = tcovering, seed = seed, verbose_result = False, 
                    decode_method = decode_method)

Pre-processing data...


Interpreting documents:   6%|▌         | 1/18 [00:14<04:02, 14.25s/it]

Document 0's Sentence segmentation performance:  {('R', 1.0), ('P', 1.0), ('F', 1.0)}
Document 0's STY accuracy:  0.6851851851851852
Document 0's Token segmentation performance without space:  {('F', 0.9302325581395349), ('P', 0.9340659340659341), ('R', 0.9264305177111717)}
Document 0's POS accuracy with/out space: 0.7701207243460765 0.5651760228353948
Document 0's SUP:DEP accuracy with/out space:  0.5628772635814889 0.19695528068506185



Interpreting documents:  11%|█         | 2/18 [00:34<04:45, 17.84s/it]

Document 1's Sentence segmentation performance:  {('R', 1.0), ('P', 1.0), ('F', 1.0)}
Document 1's STY accuracy:  0.8611111111111112
Document 1's Token segmentation performance without space:  {('F', 0.7701102490812577), ('P', 0.7165653495440729), ('R', 0.8323036187113857)}
Document 1's POS accuracy with/out space: 0.7464871194379391 0.5183537263626251
Document 1's SUP:DEP accuracy with/out space:  0.5415690866510539 0.15016685205784205



...

Document 15's Sentence segmentation performance:  {('R', 1.0), ('P', 1.0), ('F', 1.0)}
Document 15's STY accuracy:  0.8421052631578947
Document 15's Token segmentation performance without space:  {('F', 0.823529411764706), ('P', 0.7952559300873908), ('R', 0.853887399463807)}
Document 15's POS accuracy with/out space: 0.7760586319218241 0.56
Document 15's SUP:DEP accuracy with/out space:  0.5732899022801303 0.1744



Interpreting documents:  94%|█████████▍| 17/18 [03:35<00:11, 11.22s/it]

Document 16's Sentence segmentation performance:  {('R', 1.0), ('P', 1.0), ('F', 1.0)}
Document 16's STY accuracy:  0.32558139534883723
Document 16's Token segmentation performance without space:  {('F', 0.9004871259568545), ('P', 0.8850889192886456), ('R', 0.9164305949008499)}
Document 16's POS accuracy with/out space: 0.7922283901665345 0.5950540958268934
Document 16's SUP:DEP accuracy with/out space:  0.5836637589214909 0.2071097372488408



Interpreting documents: 100%|██████████| 18/18 [03:44<00:00, 12.47s/it]

Document 17's Sentence segmentation performance:  {('R', 1.0), ('P', 1.0), ('F', 1.0)}
Document 17's STY accuracy:  0.4153846153846154
Document 17's Token segmentation performance without space:  {('F', 0.9170549860205033), ('P', 0.9085872576177285), ('R', 0.9256820319849483)}
Document 17's POS accuracy with/out space: 0.8088614116434827 0.6275100401606426
Document 17's SUP:DEP accuracy with/out space:  0.5692941782586296 0.18373493975903615

Overall sentence segmentation performance:  {('R', 1.0), ('P', 1.0), ('F', 1.0)}
Overall STY accuracy:  0.6588366890380313
Overall Token segmentation performance without space:  {('F', 0.889170780134021), ('P', 0.8779611174644666), ('R', 0.9006703910614525)}
Overall POS accuracy with/out space: 0.7731257654870108 0.5660458806117414
Overall SUP:DEP accuracy with/out space:  0.5574356990910849 0.17365564874198322


In [13]:
if ptdocs:
    model.interpret(tdocs, eval_layers = test_layers, covering = tcovering, seed = seed, verbose_result = False, 
                    decode_method = decode_method)

Pre-processing data...


Interpreting documents:   6%|▌         | 1/18 [00:12<03:40, 12.98s/it]

Document 0's STY accuracy:  0.6851851851851852
Document 0's POS accuracy with/out space: 0.7852112676056338 0.5937202664129401
Document 0's SUP:DEP accuracy with/out space:  0.5699195171026157 0.2093244529019981



Interpreting documents:  11%|█         | 2/18 [00:25<03:25, 12.82s/it]

Document 1's STY accuracy:  0.8611111111111112
Document 1's POS accuracy with/out space: 0.7792740046838408 0.5806451612903226
Document 1's SUP:DEP accuracy with/out space:  0.5298594847775175 0.12458286985539488



...

Document 15's STY accuracy:  0.8421052631578947
Document 15's POS accuracy with/out space: 0.8298045602605864 0.6656
Document 15's SUP:DEP accuracy with/out space:  0.5781758957654723 0.1872



Interpreting documents:  94%|█████████▍| 17/18 [03:03<00:09,  9.74s/it]

Document 16's STY accuracy:  0.32558139534883723
Document 16's POS accuracy with/out space: 0.8112609040444092 0.6321483771251932
Document 16's SUP:DEP accuracy with/out space:  0.5900079302141158 0.22565687789799072



Interpreting documents: 100%|██████████| 18/18 [03:11<00:00, 10.66s/it]

Document 17's STY accuracy:  0.4153846153846154
Document 17's POS accuracy with/out space: 0.8397733127253992 0.6877510040160643
Document 17's SUP:DEP accuracy with/out space:  0.5790829469345699 0.20281124497991967

Overall STY accuracy:  0.6588366890380313
Overall POS accuracy with/out space: 0.7968155740346806 0.6113714849531328
Overall SUP:DEP accuracy with/out space:  0.5634951331141623 0.18494079921065615
